# CIFAR - 10 Images 

In [2]:
# Libraries
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, optimizers

# Helper Lib
import numpy as np

In [3]:
# Cifar is a set of 60K images 32 * 32 with 3 color channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)

# Constants
EPOCHS = 20
BATCH_SIZE = 128
CLASSES = 10
VALIDATION_SPLIT = 0.2
OPTIM = optimizers.RMSprop()

In [ ]:
# Build the model
